In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

## Load in the data

In [3]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 173s 1us/step


In [4]:
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [5]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


## Build the model using functional API

In [6]:
def create_model():
    i = Input(shape=x_train[0].shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(K, activation='softmax')(x)
    
    model = Model(i, x)
    return model

In [7]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2024-08-18 23:26:46.801093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-18 23:26:47.484995: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-18 23:26:47.485045: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-18 23:26:47.512745: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-18 23:26:47.512840: I external/local_xla/xla/stream_executor

In [8]:
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [9]:
with strategy.scope():
    model = create_model()

    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [10]:
# Fit
r = model.fit(x_train, y_train,
             validation_data=(x_test, y_test),
             epochs=15)

Epoch 1/15


2024-08-18 23:28:28.296563: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-18 23:28:31.546184: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff1fc2f3280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-18 23:28:31.546228: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-08-18 23:28:31.598203: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1723998511.769780    5395 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - 39s 18ms/step - loss: 1.3081 - accuracy: 0.5502 - val_loss: 1.0286 - val_accuracy: 0.6402
Epoch 2/15
1563/1563 [==============================] - 22s 14ms/step - loss: 0.8586 - accuracy: 0.7032 - val_loss: 0.7976 - val_accuracy: 0.7331
Epoch 3/15
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7043 - accuracy: 0.7557 - val_loss: 0.7121 - val_accuracy: 0.7604
Epoch 4/15
1563/1563 [==============================] - 20s 13ms/step - loss: 0.5927 - accuracy: 0.7961 - val_loss: 0.8426 - val_accuracy: 0.7220
Epoch 5/15
1563/1563 [==============================] - 21s 14ms/step - loss: 0.5090 - accuracy: 0.8248 - val_loss: 0.6501 - val_accuracy: 0.7851
Epoch 6/15
1563/1563 [==============================] - 20s 13ms/step - loss: 0.4334 - accuracy: 0.8515 - val_loss: 0.6471 - val_accuracy: 0.7940
Epoch 7/15
1563/1563 [==============================] - 19s 12ms/step - loss: 0.3623 - accuracy: 0.8746 - val_loss: 0.6222 - val_accura

In [11]:
50000/391

127.8772378516624

In [12]:
10000/79

126.58227848101266

In [13]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 25s 13ms/step - loss: 1.3101 - accuracy: 0.5482 - val_loss: 1.0191 - val_accuracy: 0.6504
Epoch 2/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8378 - accuracy: 0.7094 - val_loss: 0.8859 - val_accuracy: 0.7005
Epoch 3/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.6889 - accuracy: 0.7627 - val_loss: 0.7231 - val_accuracy: 0.7543
Epoch 4/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5816 - accuracy: 0.7992 - val_loss: 0.6485 - val_accuracy: 0.7812
Epoch 5/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.4939 - accuracy: 0.8304 - val_loss: 0.6659 - val_accuracy: 0.7811
